# Check monotonicity of results
This notebook is not an EDA notebook but rather a tool to check if the results in the vehicle_data table are monotonically decreasing.  

In [ ]:
from transform.raw_results.odometer_aggregation import agg_last_odometer
from transform.raw_results.ford_results import get_results as ford_results
from transform.raw_results.tesla_results import get_results as tesla_results
from transform.raw_results.volvo_results import get_results as volvo_results
from transform.raw_results.renault_results import get_results as renault_results
from transform.raw_results.mercedes_results import get_results as mercedes_results

### Data extraction

In [ ]:
def get_all_result() :
    df = agg_last_odometer('kia', force_update=False)
    df = pd.concat((df, agg_last_odometer('bmw', force_update=False)))
    df = pd.concat((df, ford_results(force_update=False)))
    df = pd.concat((df,tesla_results(force_update=False)))
    df = pd.concat((df,volvo_results(force_update=False)))
    df = pd.concat((df,renault_results(force_update=False)))
    df = pd.concat((df,mercedes_results(force_update=False)))
    return df

In [ ]:
result = get_all_result()

### Check monoticity

In [ ]:
def soh_is_monotonically_decreasing(df):
    return df.set_index("date")["soh"].is_monotonic_decreasing

montonic_mask_per_vin = (
    result.dropna(subset=['soh'])
    .groupby("vin")
    .apply(soh_is_monotonically_decreasing, include_groups=False)
    .to_frame("monotonically_decreasing")
    .reset_index(drop=False)
)


In [ ]:
value_counts_per_vin = {
    "normalized": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=True),
    "absolute": montonic_mask_per_vin["monotonically_decreasing"].value_counts(dropna=False, normalize=False)
}
pd.concat(value_counts_per_vin, axis=1, keys=value_counts_per_vin.keys(), names=["value_counts_type"])